# Intro

The initial dataset being sourced is from the HuggingFace library, (https://huggingface.co/datasets/brianarbuckle/cocktail_recipes)

This initial dataset contains columns `title`, `ingredients`, `directions`, `misc`, `source`, and `ner`.

**Overview**
The 'Ingredients' column of our dataset presents a unique challenge. It consists of objects, each being a list of strings, where each string details a part of a cocktail recipe. These details typically include the quantity, unit, and name of an ingredient, but they can also contain preparation instructions or garnishing details that are not directly usable in their current form for data analysis or recipe generation.

**Objective**
Our goal is to parse these strings to extract structured information that can be effectively utilized in our project. Specifically, we aim to separate the ingredient details into distinct components: quantity, unit, and ingredient name, while filtering out the non-ingredient related information.

## Imports

In [7]:
import pandas as pd
import re
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns

## Initial Data Inspection

In [8]:
# Loading the cocktail_recipes dataset
dataset = load_dataset('brianarbuckle/cocktail_recipes')

df = pd.DataFrame(dataset['train'])

In [9]:
df.head(5)

,title,ingredients,directions,misc,source,ner
0,151 Swizzle,[1.5 oz. 151-Proof Demerara Rum [Lemon Hart or...,[],[],Beachbum Berry Remixed,"[pernod, rum]"
1,20th Century,"[The 21st Century, 2 oz. Siete Leguas Blanco T...","[shake on ice and strain into coupe , The Best...",[],Jim Meehan,"[cocchi americano, pernod, tequila]"
2,20th Century,"[1.5 oz. Plymouth Gin, 3\/4 oz. Mari Brizard W...",[shake on ice and strain],[],PDT,"[lillet, gin]"
3,Abbey Cocktail,[],"[Shake liquid ingredients with ice., Strain in...",[Suggested glassware is Cocktail Glass],The Ultimate Bar Book,[]
4,Absinthe Drip,[1 1/2 ounces Pernod (or other absinthe substi...,[Pour Pernod into a pousse-caf or sour glass....,[The Absinthe Drip was made famous by Toulouse...,The Ultimate Bar Book,"[pernod, absinthe]"


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 875 entries, 0 to 874
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        875 non-null    object
 1   ingredients  875 non-null    object
 2   directions   875 non-null    object
 3   misc         875 non-null    object
 4   source       875 non-null    object
 5   ner          875 non-null    object
dtypes: object(6)
memory usage: 41.1+ KB


In [11]:
df.shape

(875, 6)

# Data Cleaning

## Lowercase standardization

## Deleting Empty Rows

### Filtering rows where `Ingredients` is empty

There were two scenarios in the data, ingredients being len of 0, and ingredients being an empty string list (len of 1, being the list, but its still empty).

In [12]:
# Filter rows where `ingredients` is not empty and not just an empty string list
modified_df = df[df['ingredients'].apply(lambda x: len(x) > 0 and x != [''])].reset_index(drop=True)

### Removing Irreparable Entries

This section will involve deleting entries which we have deemed problematic and unfixable, associated reason for deletion will be in the comment.

In [13]:
indices_to_drop = [7] # Industrial size recipie, acts as outlier for portions 

In [14]:
# We reset index AFTER dropping all the `indices_to_drop` , as we must retain the indexing while dropping all of them, otherwise dropping them will affect the index of the subsequently dropped recipie
modified_df = modified_df.drop(index=indices_to_drop).reset_index(drop=True) 

## Normalizing Ingredients Lists: Separating Combined Strings

In [18]:
def split_combined_ingredients(ingredients_list):
    """
    Splits a single-item list containing a string of ingredients separated by commas
    into a list of individual ingredients. If the list contains more than one item or
    if the single item does not contain a comma, the original list is returned.

    This is useful for normalizing ingredients lists where all ingredients are combined
    into a single string.

    Parameters:
    - ingredients_list: A list of ingredient(s).

    Returns:
    - A list of separated and trimmed ingredient strings.
    """
    if len(ingredients_list) == 1 and ',' in ingredients_list[0]:
        return [ingredient.strip() for ingredient in ingredients_list[0].split(', ')]
    else:
        return ingredients_list

# Applying the function
modified_df['ingredients'] = modified_df['ingredients'].apply(split_combined_ingredients)

## Normalizing: Deleting ingredients listed as optional 

In [19]:
def filter_optional_ingredients(ingredients_list):
    """
    Filters out ingredients marked as "optional" from a list of ingredients.
    
    This function iterates through a given list of ingredient strings, removing
    any ingredient that contains the word "optional". The goal is to create a more
    concise and essential list of ingredients by excluding those that are not
    crucial to the recipe. This step is part of data cleaning to standardize
    the ingredients data for further analysis.
    
    Parameters:
    - ingredients_list: A list of strings, where each string is an ingredient.

    Returns:
    - A list of ingredient strings, excluding any marked as "optional".
    """
    optional_keyword = 'optional'
    filtered_list = [ingredient for ingredient in ingredients_list if optional_keyword not in ingredient.lower()]
    return filtered_list

# Apply the filtering function to the 'ingredients' column
modified_df['ingredients'] = modified_df['ingredients'].apply(filter_optional_ingredients)


# Parsing

Actual Parsing Function here

# Post-Parsing Analysis

## Tools to Inspect

### `inspect_row` function

### none_rows call

### none_rows.loc[none_rows[’ingredient_step’] == 0]  → allows us to see where ingredient_step =0

### Future tools/calls to look into problems

Can leave this empty for now, just putting in a space here for futureproofing the cleanliness

## Visual Analysis

### Graphs/Visuals